In [1]:
import requests
import json

In [2]:
class URLMaker:
    #api 기본 주소세팅. 뒤에 쿼리 덧붙여서 명령별 구별 
    url = 'https://api.themoviedb.org/3'

    def __init__(self, key):
        self.key = key

    #영화/추천영화 긁어오기 - GET movie/popular (https://developers.themoviedb.org/3/movies/get-popular-movies)
    # 한페이지만.
    def get_movie_url(self, category='movie', feature='popular', page='1'):
        #위의 url(자기자신 소속 url 데이터 line 6)/movie/popular 로 기능 구동
        url = f'{self.url}/{category}/{feature}'
        #뒤에 쿼리로 인증용 필수 데이터 api_key(line 8), 옵션 데이터인 언어(한글-한국), 페이지수(1페이지) 덧붙임
        url += f'?api_key={self.key}&language=en-US&page={str(page)}'

        #그렇게 만든 최종 명령 url 반환
        return url

    def movie_id(self, title):
        #키와 url, 받아온 타이틀 정보 보유.
        #
        url = self.get_url('search', 'movie', region='KR', language='ko', query=title)
        #requests.get
        res = requests.get(url)
        #위에서 요청받은 json(딕셔너리 타입)으로 받아와 movie에 저장
        movie = res.json()

        # movie 딕셔너리 군에 results키에 대응하는 밸류값이 있다면(0이 아니라면) 첫번째 밸류의 id에 대응되는 값. 그 외엔 빈칸이니 id 반환하지 않음.
        if len(movie.get('results')):
            return movie.get('results')[0].get('id')
        else:
            return None

    def get_genre_url(self):
        url = f'{self.url}/genre/movie/list?api_key={self.key}'
        return url

In [3]:
#인증키
TMDB_KEY = '234c446afa7c9156e6c9a101b724f5ae'
#인증키를 넣고 돌리면 반환되는 최종 url
url = URLMaker(TMDB_KEY)

def create_genre_data():
    #line 37에서 반환한 url값 받아와 저장
    genre_url = url.get_genre_url()
    #리퀘스트 데이터 저장
    raw_data = requests.get(genre_url)
    #해당 데이터 json화
    json_data = raw_data.json()
    #시킨것에서 genres라는 키의 밸류값 추출
    genres = json_data.get('genres')

    #장르 데이터 저장용 빈 리스트 생성. 여기에 딕셔너리 하나씩 넣을 예쩡
    genre_data = []

    #genres의 모든 원소들을 하나하나 tmp에 저장한뒤 genre_data에 하나하나 추가함
    for genre in genres:
        tmp = {
            #나중에 찾아오려는 심산인지 모델명을 movies.genre로 통일시켜둠
            'model': 'movies.genre',
            'pk': genre['id'],
            'fields': {
                'name': genre['name']
            }
        }
        genre_data.append(tmp)

    #모든 moives의 movie들을 다 돌려 원하는 내용만 만들어 추가한 딕셔너리 덩어리 genre_data 다 만든 뒤,
    #tmdb.json을 쓰기전용 사양('w')으로 오픈. 없다면 해당 빈파일 생성하고 있다면 싹 비우고 내용입력 시작.    
    with open('tmdb.json', 'w') as f:
        #파이썬 객체를 json문자열로 변환. 위에 '열어둔' tmdb.json(이하 f)에 genre_data 파일 싹 입력. 
        #indent : 들여쓰기 옵션. 보통 4로 놓고쓰며 줄바꿈 잘 되고 이쁘게 된다.
        #load와 마찬가지로 파이썬 내부에 놓고 쓰려면 dumps쓰면 되는데 잘 쓰진 않는것같다.
        json.dump(genre_data, f, indent=4)


In [4]:
def create_movie_data():
    #'tmdb.json' 읽어오기. r+는 읽기 또는 쓰기모드. 밑에 w도 있는데 w는 여는순간(없다면 새로 만들어서라도) 안의 내용물 싹 밀어버리고 기록하는 타입이라면
    # r+는 내용물은 건드리지 않고 이어 기록하는 타입. 따라서 수정할거면 무조건 w를 써야하지만 추가만 할거면 r+가 안전하다. 이하 f로 약칭.
    with open('tmdb.json', 'r+') as f:
        #json.load()로 외부의 json파일(여기선 open한 tmdb)을 파이썬 객체화 시킨놈이 movie_data (디코딩)
        #json.loads()도 있는데 이건 쉽게말해서 파이썬 내부 json타입 데이터스트링을 json 오브젝트로 만들때 쓴다고 보면 편하다 (인코딩)
        movie_data = json.load(f)

    for page in range(1, 501):
        #1부터 500까지의 '페이지 번호'에 맞춰 get_movie_url(line 13) 가동. 그래서 get_movie_url를 1페이지씩 불러오게 시켰던것
        raw_data = requests.get(url.get_movie_url(page=page))
        #그렇게 불러온 외부 데이터 raw_data를 파이썬 내부로. json_data. 타입:dictionary
        json_data = raw_data.json()
        #json_data의 results 키값을 갖는 밸류값들만 토해서 movies에 저장. 타입:list
        movies = json_data.get('results')

        for movie in movies:
            #movies의 각 원소들에 대해 그 원소들과 대응하는 데이터 추출. 
            #단, realease_date, poster_path에 대응하는 밸류값이 없을경우 무시하고 넘어감
            if movie.get('release_date') == "" or movie.get('poster_path') == "":
                continue

            #.pop() 리스트의 마지막 데이터를 뽑아내고 삭제함. 
            #한개 입력되어있을 backdrop_path 반환하고 movie에서 삭제
            ##민구가 필요하다고 한것같으니 임시 비활성화함
            #movie.pop('backdrop_path')
            
            #video에서도 하나 뽑아내어 반환하고 movie에서 삭제
            movie.pop('video')
            #like_users에 들어있는 리스트 초기화
            movie['like_users'] = []
            movie['movie_reference_genre'] = []
            movie['movie_reference_overview'] = []

            #tmp 딕셔너리 초기화. model
            tmp = {
                'model': 'movies.movie',
                'pk': movie.pop('id'),
                'fields': movie,
            }
            #그렇게 만든 tmp를 movie_data에 '추가' 아까 line72에서 로드한 파일에 추가.
            movie_data.append(tmp)

    #모든 moives의 movie들을 다 돌려 원하는 내용만 만들어 추가한 딕셔너리 덩어리 movie_data 다 만든 뒤,
    #tmdb.json을 쓰기전용 사양('w')으로 오픈. 없다면 해당 빈파일 생성하고 있다면 싹 비우고 내용입력 시작.
    with open('tmdb.json', 'w') as f:
        #파이썬 객체를 json문자열로 변환. 위에 '열어둔' tmdb.json(이하 f)에 movie_data 파일 싹 입력. 
        #indent : 들여쓰기 옵션. 보통 4로 놓고쓰며 줄바꿈 잘 되고 이쁘게 된다.
        #load와 마찬가지로 파이썬 내부에 놓고 쓰려면 dumps쓰면 되는데 잘 쓰진 않는것같다.
        json.dump(movie_data, f, indent=4)

In [5]:
#
#create_genre_data()
create_movie_data()

In [ ]:
#tmdb.json 영화 데이터 표본
{
        "model": "movies.movie",
        "pk": 127585,
        "fields": {
            "adult": false,
            "backdrop_path": "/hUPgIibqZlwbhs4N08cPzzc4f5K.jpg",
            "genre_ids": [
                28,
                12,
                14,
                878
            ],
            "original_language": "en",
            "original_title": "X-Men: Days of Future Past",
            "overview": "\ub3cc\uc5f0\ubcc0\uc774\ub4e4\uc744 \uba78\uc885\uc2dc\ud0a4\uae30 \uc704\ud574 \uc81c\uc791\ub41c \ub85c\ubd07 \uc13c\ud2f0\ub12c\uc774 \uc778\uac04\ub4e4\uae4c\uc9c0 \ubaa8\uc870\ub9ac \ub9d0\uc0b4\ud558\ub294 2023\ub144. \uc778\ub958\uc758 \uc885\ub9d0\uc774 \ucf54\uc55e\uc5d0 \ub2e5\uce5c \uc0c1\ud669\uc5d0\uc11c \ud504\ub85c\ud398\uc11cX\uc640 \ub9e4\uadf8\ub2c8\ud1a0\ub294 \uacfc\uac70\ub97c \ud1b5\ud574 \ubbf8\ub798\uc758 \uc6b4\uba85\uc744 \ubc14\uafb8\uace0\uc790 \ud55c\ub2e4. \uc6b8\ubc84\ub9b0\uc740 \ud2b8\ub77c\uc2a4\ud06c\uac00 \uc13c\ud2f0\ub12c\uc744 \uac1c\ubc1c\ud558\ub294 \uac83\uc744 \ub9c9\uae30 \uc704\ud574 1973\ub144\uc73c\ub85c \ubcf4\ub0b4\uc9c4\ub2e4. 1973\ub144\uc758 \ucc30\uc2a4\ub294 \uc808\ub9dd\uc5d0 \ube60\uc9c4 \ucc44 \ud589\ud06c \ub9e5\ucf54\uc774\uc640 \ud568\uaed8 \uc740\ub454 \uc0dd\ud65c\uc744 \ud558\uace0 \uc788\uace0, \uc5d0\ub9ad\uc740 \ucf00\ub124\ub514 \ub300\ud1b5\ub839 \uc554\uc0b4 \uc6a9\uc758\uc790\ub85c \ubd99\uc7a1\ud600 \ud39c\ud0c0\uace4 \uc9c0\ud558 \uae4a\uc219\ud55c \uacf3\uc5d0 \uc704\uce58\ud55c \uc218\uc6a9\uc18c\uc5d0 \uac07\ud600 \uc788\ub2e4.",
            "popularity": 64.747,
            "poster_path": "/zm5zanw86TX2T7qJNTKilL3x3S8.jpg",
            "release_date": "2014-05-15",
            "title": "\uc5d1\uc2a4\ub9e8: \ub370\uc774\uc988 \uc624\ube0c \ud4e8\ucc98 \ud328\uc2a4\ud2b8",
            "vote_average": 7.5,
            "vote_count": 12889,
            "like_users": []
            
        }
    }